# 淘宝用户行为分析

## 项目简介
随着移动互联网技术的飞速发展和大众人均收入的增长，电商行业也随着技术的发展迅速崛起又快速分化。在当下大众消费需求日趋多样化的时代背景下，移动互联网行业特别是电商行业不再依靠用户红利实现业务增长，开始从粗放型的经营模式转向精细化管理，需要结合市场、渠道、用户行为等数据分析，对用户展开有针对性的运营活动，提供个性化、差异化的经营策略，从而实现运营目的。  
本项目使用**MySQL**分析淘宝平台2017年11月25日至2017年12月3日的用户行为数据，通过对网站流量、用户活跃率、用户行为转化漏斗、用户行为偏好等特征的分析，提供有针对性的运营策略。

## 数据理解

### 数据来源

本项目数据来源于阿里天池[User Behavior Data](https://tianchi.aliyun.com/dataset/dataDetail?dataId=649)，为了快速分析项目模型，项目仅截取完整数据集`UserBehavior`(3.41GB)的部分数据记录`UB_split_10.csv`(171.7MB)作为本项目的数据集。

### 数据属性
<table> 
    <tr>
        <td>
            <table border="1" cellspacing="5" cellpadding="5" align="left" style="text-align:left">
                <tr>
                    <th>变量名</th>
                    <th>中文释义</th>
                    <th>数据类型</th>
                </tr>    
                <tr>
                    <td >UserID</td>
                    <td>用户ID</td>
                    <td>整型</td>
                </tr>
                <tr>
                    <td>ItemID</td>
                    <td>商品ID</td>
                    <td>整型</td>
                </tr>
                <tr>
                    <td>CategoryID</td>
                    <td>商品所属种类ID</td>
                    <td>整型</td>
                </tr>
                <tr>
                    <td>CategoryID</td>
                    <td>商品所属种类ID</td>
                    <td>整型</td>
                </tr>
                <tr>
                    <td>BehaviorType</td>
                    <td>行为类型，包含pv,cart,buy和fav四种类型的行为</td>
                    <td>字符串</td>
                </tr>
                <tr>
                    <td>Timestamp</td>
                    <td>时间戳，用户发生某项行为当下时间点，精确到秒</td>
                    <td>整型</td>
                </tr>
            </table>
        </td>
     </tr>
    <tr> 
        <td>
            <table border="1" cellspacing="5" cellpadding="5" align="left" style="text-align:left">
                <tr>
                    <th>用户行为</th>
                    <th>解释说明</th>
                </tr>
                <tr>
                    <td>pv</td>
                    <td>商品浏览</td>
                </tr>
                <tr>
                    <td>cart</td>
                    <td>加入购物车</td>
                </tr>
                <tr>
                    <td>fav</td>
                    <td>收藏</td>
                </tr>
                <tr>
                    <td>buy</td>
                    <td>购买</td>
                </tr>
            </table>
        </td>
    </tr>
</table>

## 评估指标
<img src='.\评估指标.png'>

## 提出问题
根据上述的业务评估指标，项目分析主要解决以下问题：

- Q1. 网站流量情况：访问量PV、独立访客数UV、人均访问数PV/UV、跳出率（只存在浏览行为的访问量/PV）、DAU、活跃率等指标情况；
- Q2. 探索用户从浏览到购买整个过程的转化或者流失情况，从而确定关键夹点位置，为后续的改进提出意见；
- Q3. 探索用户的行为模式，从日期、时间、商品种类销量、具体商品销量等维度探索用户的行为偏好和模式；
- Q4. 挖掘用户价值，发现高价值用户

# 数据导入

使用ETL工具Kettle快速读写数据集`UB_split_10.csv`到数据库中，因为拆分的数据集没有字段名，所以预先在数据库中创建表结构和编码规则

```mysql
CREATE TABLE IF NOT EXISTS user_behavior_split_10 (
UserID INT NOT NULL,
ItemID INT,
CategoryID INT,
BehaviorType VARCHAR(255),
Timestamp INT) ENGINE INNODB DEFAULT CHARSET=utf8;
```

# 数据评估与清洗
对数据集的基本信息，如数据类型、重复值、缺失值、异常值等评估和清理

## 观察记录

```Mysql
-- 查看总记录数
SELECT count(*) AS 总记录数
FROM user_behavior_split_10

-- 预览数据
SELECT * 
FROM user_behavior_split_10
LIMIT 10
```
本项目的数据集共有500万条记录，每一条记录代表一个用户的用户行为

![image.png](.\sql-results-pngs\scan-data-before-clean.png)

## 评估数据


- 查看各个变量的数据类型

```Mysql
SELECT COLUMN_NAME, DATA_TYPE
FROM information_schema.`COLUMNS`
WHERE TABLE_NAME='user_behavior_split_10' AND TABLE_SCHEMA = 'userbehavior_taobao';
```
![](.\sql-results-pngs\datatype-before-clean.png)

- 查看Behavior Type的唯一值

```Mysql
SELECT DISTINCT BehaviorType
FROM user_behavior_split_10
```

![](.\sql-results-pngs\unique-value-UBT.png)




> 部分变量的数据类型存在错误：
- UserID、ItemID的数据类型应改为字符串
- Timestamp需改为标准可读的日期时间形式

## 数据类型处理

- 更正数据类型和新建日期时间相关列

```Mysql
-- 将UserID和ItemID的数据类型改为字符串
ALTER TABLE user_behavior_split_10
MODIFY COLUMN UserID VARCHAR(100),
MODIFY COLUMN ItemID VARCHAR(100);

-- 根据时间戳新建datetime，date（天）和小时列

ALTER TABLE user_behavior_split_10 ADD Datetime TIMESTAMP(0) NULL;
UPDATE user_behavior_split_10
SET Datetime = FROM_UNIXTIME(user_behavior_split_10.`Timestamp`);
 
ALTER TABLE user_behavior_split_10 ADD Date CHAR(10) NULL
UPDATE user_behavior_split_10
SET Date = SUBSTR(Datetime,1,10);

ALTER TABLE user_behavior_split_10 ADD Hour char(2) NULL
UPDATE user_behavior_split_10
SET Hour = SUBSTR(Datetime,12,2);
```
![](.\sql-results-pngs\scan-data-after-clean.png)


- 查看更改之后的数据类型

```Mysql
SELECT COLUMN_NAME,DATA_TYPE
FROM information_schema.`COLUMNS`
WHERE TABLE_NAME='user_behavior_split_10' AND TABLE_SCHEMA= 'userbehavior_taobao'
```
![](.\sql-results-pngs\datatype-after-clean.png)



## 异常值处理

- 探索行为发生的时间是否存在不在日期时间范围内或空值的数据记录

```Mysql
SELECT *
FROM user_behavior_split_10
WHERE Datetime < '2017-11-25 00:00:00' OR Datetime >= '2017-12-04 00:00:00' OR Datetime IS NULL

```  
![](.\sql-results-pngs\outoftime-records.png)

- 删除时间范围外的数据记录和时间日期为空值的数据记录

```Mysql
BEGIN;
DELETE FROM user_behavior_split_10
WHERE Datetime < '2017-11-25 00:00:00' OR Datetime >= '2017-12-04 00:00:00' OR Datetime IS NULL;
COMMIT;

-- 查看是否删除完成 
SELECT *
FROM user_behavior_split_10
WHERE Datetime < '2017-11-25 00:00:00' OR Datetime >= '2017-12-04 00:00:00' OR Datetime IS NULL

```
![](.\sql-results-pngs\del-outoftime-records.png)



# 探索性分析
通过可视化的方式，探索11月25日-12月3日期间，淘宝用户在访问网站、行为转化率、用户行为偏好、商品销量等现状探索

## 网站流量情况

### PV、UV、人均访问数
$PV = 浏览商品的累计次数$  
$UV = 访问网站的不重复用户个数$  
$人均访问数 = \frac{PV}{UV}$

```Mysql
-- 将流量结果用table封装以便后续探索的使用
CREATE TABLE web_traffic AS
SELECT t1.*, t1.PV/t1.UV AS '平均页面访问量'
FROM (SELECT COUNT(DISTINCT UserID) AS 'UV',
 (SELECT COUNT(*)
	FROM user_behavior_split_10
	WHERE BehaviorType = 'pv') AS 'PV'
FROM user_behavior_split_10) AS t1
SELECT *
FROM web_traffic

```
![](.\sql-results-pngs\web-traffic.png)

本期间内，有48,503位用户访问了淘宝，并产生了4,469,990条浏览记录，人均访问量约为92条

### 跳出率、日均活跃用户数、活跃率
$跳出率 = \frac{只有点击行为的用户数}{总用户数} $  
$日均活跃用户数(DAU) = \frac{\sum{日UV}}{天数}$  
$日活率(DAU Rate) =\frac{DAU}{UV}$

- 跳出率
```Mysql
SELECT (UV - (SELECT COUNT(DISTINCT UserID)
FROM user_behavior_split_10
WHERE BehaviorType != 'pv'))/UV AS '跳出率'
FROM web_traffic;
```
![](.\sql-results-pngs\bounce-rate.png)

- 日均活跃用户数和活跃率
```Mysql
SELECT t3.`日均活跃用户数`, t3.`日均活跃用户数`/web_traffic.UV AS '日活率'
FROM (SELECT AVG(t2.unique_user_count) AS '日均活跃用户数'
FROM (SELECT Date,COUNT(DISTINCT UserID) AS 'unique_user_count' 
FROM user_behavior_split_10
GROUP BY Date) AS t2) AS t3, web_traffic
```
![](.\sql-results-pngs\DAU.png)

本周期内，淘宝跳出率约为6%，日均活跃用户数约为38,047个用户，约占该期间内总用户数的79%

### 小结
2017年11月25日至2017年12月3日期间内，有48,283位用户访问了淘宝，并产生了4,469,990条浏览记录，人均访问量约为92条。在这48,283位用户中，平均每天有79%的用户会登陆淘宝进行商品选购。淘宝网具有较大的流量规模和用户基数，高日活量和低跳出率说明淘宝具有较好的用户体验和较强的用户粘性，人们能在淘宝上解决其购物需求，其俨然成为人们的主要购物平台。

## 用户行为路径转化漏斗
探索用户从进入网站、浏览、选择加购到完成购买整个过程的转化情况，从而确定关键夹点位置，为后续的运营策略提出较有针对性的建议。

### 行为次数转化率
探索商品的整个选购过程中，每一个环节到下一个环节的行为次数转化率

- 统计各个行为的总次数

```Mysql
-- 插入cart和fav的总计
INSERT INTO bh_total_counts VALUES ('cart&fav',(SELECT
	SUM(t1.bh_counts)
FROM
	(
		SELECT
			bh_counts
		FROM
			bh_total_counts
		WHERE
			BehaviorType IN ('cart', 'fav')
	) AS t1))
                                                
                                                
-- 删除cart和fav的记录

DELETE FROM bh_total_counts
WHERE BehaviorType in ('fav','cart')
                                                                                      
```
![](.\sql-results-pngs\bh-counts.png)

> 因为加入购物车和收藏在购买商品的步骤上没有先后顺序，而且两种行为都能表现用户对产品可能存在购买意向，同属于购买意向确认阶段，所以可以将这两种行为合并作为购买意向确认阶段的行为总次数；

> 加入购买车或收藏也并非进入购买页面的必要行为，因为部分用户也可以从浏览商品页面直接进入购买页面，但这并不影响计算最后的整体转化率

- 行为总数转化漏斗  
> 使用PowerBI可视化行为总数转化漏斗

![](.\sql-results-pngs\bhcounts-converse-rate.png)

从上图可知，仅有9.56%的浏览行为转化为加入购物车或收藏行为，从加入购物车和收藏商品到购买的转化率为23.07%，总体的购买转化率为2.2%。从整个选购过程的转化率来看，由浏览转化为加入购物车或收藏行为是整个过程中行为流失量最多的环节，所以该阶段应为整个过程的关键夹点，需要在此环节提高转化率

### 不同行为的独立用户人数转化率

- 统计不同行为的独立用户人数

```Mysql
SELECT
```

## 用户行为偏好探索
探索用户的行为模式，从时间日期、商品种类等因素探索用户的行为偏好和模式

### 日期时间维度

- 按照日期分组，计算每一天的PV和UV
```Mysql
CREATE TABLE bh_date AS 
SELECT Date, COUNT(*) AS "PV", COUNT(DISTINCT UserID) AS "UV"
FROM user_behavior_split_10
WHERE BehaviorType = "pv"
GROUP BY Date
```
- 导出表bh_date，可视化每一天的PV和UV情况


- 按照时间点分组，计算每一个时刻的平均PV和UV

```Mysql
CREATE TABLE bh_hour AS 
SELECT `Hour`, COUNT(*) AS "PV", COUNT(DISTINCT UserID) AS "UV"
FROM user_behavior_split_10
WHERE BehaviorType = "pv"
GROUP BY `Hour

-- 计算平均每天该时刻的APV和AUV
ALTER TABLE bh_hour
ADD COLUMN APV DECIMAL(10,2),
ADD COLUMN AUV DECIMAL(10,2);
UPDATE bh_hour
SET APV = PV/(SELECT COUNT(*) FROM bh_date),AUV = UV/(SELECT COUNT(*) FROM bh_date);
```

- 导出表bh_date，可视化每一天的PV和UV情况

### 商品种类维度

- 不同的商品种类之间是否存在较大的购买量差异？如果存在较大差异，继续探索下面三个问题。
- 哪些种类的商品兼具高浏览量和高购买量？
- 哪些种类的商品虽然没有较高的活跃度但是付费成功率较高，购买率较高？
- 哪些种类的商品虽然有较高的活跃度，但是付费成功率较低，购买率较低？